In [1]:
%%capture
import sys
!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension
!pip install datasets
!pip install transformers
!pip install multiprocessing
!pip install evaluate

In [2]:
from transformers import BertForSequenceClassification, AutoTokenizer
from datasets import load_dataset
import pandas as pd
import evaluate
import torch
from transformers import get_scheduler
import numpy as np
from torch.optim import AdamW
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
dataset = load_dataset("imdb")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Модель до дооубучения
Посмотрим, как она работает для задачи классификации. Для этого был взят датасет для бинарной классификации отзывов фильмов с двумя классами: позитивный комментарий (1) и негативный (0). Без обучений модель на метрике recall показала результат 1, поэтому решено было повысить метрику accuracy = 0.5. (f1 было решено не брать для повышения, так как классы являются сбалансированными. на начальном этапе данная мметрика показа результат в 0.667)

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [6]:
data_train_dic = dataset['train']
data_test_dic = dataset['test']
data_train_dic[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [7]:
data_train = pd.DataFrame({'text': data_train_dic['text'], 'label': data_train_dic['label']})
data_test = pd.DataFrame({'text': data_test_dic['text'], 'label': data_test_dic['label']})

In [8]:
data_train.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [9]:
metric = evaluate.load("accuracy")

In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
def encode(docs):
    encoded_dict = tokenizer.batch_encode_plus(docs, add_special_tokens=True, max_length=128, padding='max_length',
                                               return_attention_mask=True, truncation=True, return_tensors='pt')
    input_ids = encoded_dict['input_ids']
    attention_masks = encoded_dict['attention_mask']
    return input_ids, attention_masks

In [12]:
train_input_ids, train_att_masks = encode(data_train['text'].values.tolist())
test_input_ids, test_att_masks = encode(data_test['text'].values.tolist())

In [13]:
BATCH_SIZE = 16

train_y = torch.LongTensor(data_train['label'].values.tolist())
test_y = torch.LongTensor(data_test['label'].values.tolist())

train_tensor_dataset = TensorDataset(train_input_ids, train_att_masks, train_y)
train_dataloader = DataLoader(train_tensor_dataset, shuffle=True, batch_size=BATCH_SIZE)

test_tensor_dataset = TensorDataset(test_input_ids, test_att_masks, test_y)
test_dataloader = DataLoader(test_tensor_dataset, batch_size=BATCH_SIZE)


In [14]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [15]:
model.eval()
with torch.no_grad():
    for step_num, batch_data in tqdm(enumerate(test_dataloader)):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids=input_ids, attention_mask=att_mask, labels=labels)
        predictions = torch.argmax(output.logits, dim=-1)
        metric.add_batch(predictions=predictions, references=labels)

metric.compute()

0it [00:00, ?it/s]

{'accuracy': 0.5}

# Дообучение модели
Дообучение производилось на загруженном ранее датасете. После обучения модель была аналогичным образом проверена на score. Удалось повысить значение метрики accuracy до 0.88252

In [17]:
num_epochs = 3
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [18]:
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch_data in train_dataloader:
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        outputs = model(input_ids=input_ids, attention_mask=att_mask, labels=labels)
    	
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/4689 [00:00<?, ?it/s]

In [19]:
model.eval()
with torch.no_grad():
    for step_num, batch_data in tqdm(enumerate(test_dataloader)):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids=input_ids, attention_mask=att_mask, labels=labels)
        predictions = torch.argmax(output.logits, dim=-1)
        metric.add_batch(predictions=predictions, references=labels)

metric.compute()

0it [00:00, ?it/s]

{'accuracy': 0.88252}